In [1]:
# Step 0. Load libraries and custom modules
# Matrices and dataframes
import numpy as np
import pandas as pd
# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
# Data pre-processing
